In [1]:
from pathlib import Path
import numpy as np
import pandas as pd

In [156]:
'MSG	5511177 THRESHOLDS L 52 179  R 49 184'.find('L')

19

In [178]:
line = 'THRESHOLDS L 52 179  R 49 184'
info = line.split('THRESHOLDS ')[1]
#info[info.find('L '):]
info.split(' R ')[1].split()

['49', '184']

In [186]:
mode = {'CR':'corneal reflection', 'P':'pupil'}

filter_type = {'0': 'filter off',
               '1': 'standard filter',
               '2': 'extra filter'}

mount= {'MTABLER': 'Desktop, Stabilized Head, Monocular',
        'BTABLER': 'Desktop, Stabilized Head, Binocular/Monocular',
        'RTABLER': 'Desktop (Remote mode), Target Sticker, Monocular',
        'RBTABLER': 'Desktop (Remote mode), Target Sticker, Binocular/Monocular',
        'ABRTABLE': 'I think Desktop (Remote mode), Target Sticker, Binocular/Monocular',
        'AMTABLER': 'Arm Mount, Stabilized Head, Monocular',
        'ARTABLER': 'Arm Mount (Remote mode), Target Sticker, Monocular',
        'BTOWER': 'Binocular Tower Mount, Stabilized Head, Binocular/Monocular',
        'TOWER': 'Tower Mount, Stabilized Head, Monocular',
        'MPRIM': 'Primate Mount, Stabilized Head, Monocular',
        'BPRIM': 'Primate Mount, Stabilized Head, Binocular/Monocular',
        'MLRR': 'Long-Range Mount, Stabilized Head, Monocular, Camera Level',
        'BLRR': 'Long-Range Mount, Stabilized Head, Binocular/Monocular, Camera Angled'}


def _get_header(asc_fname):
    header_info = {}
    is_header = False
    for line in Path(asc_fname).open():
        if line.startswith('**'):
            is_header = True
        else:
            return header_info
        if is_header:
            hdr = line.lstrip('** ').split(':',maxsplit=1)
            if hdr and hdr[0].isupper():
                header_info[hdr[0]] = hdr[1].strip()
                
def _get_data_spec(asc_fname, mode=mode, filter_type=filter_type, mount=mount):
    data_spec = {}
    with Path(asc_fname).open() as file:
        is_data_spec = False
        for line in file:
            if line.isspace():
                continue
            if 'RECCFG' in line:
                is_data_spec = True
            if is_data_spec:
                if 'RECCFG' in line:
                    info = line.split('RECCFG')[1].split()
                    data_spec['tracking_mode'] = mode[info[0]]
                    data_spec['srate'] = int(info[1])
                    data_spec['sample_filter'] = filter_type[info[2]]
                    data_spec['analog_filter'] = filter_type[info[3]]
                    data_spec['eyes_tracked'] = info[4]
                elif 'ELCLCFG' in line:
                    info = line.split('ELCLCFG')[1].split()
                    data_spec['mount_config'] = mount[info[0]]
                elif 'GAZE_COORDS' in line:
                    line.find('L')
                    info = line.split('GAZE_COORDS')[1].split()
                    data_spec['pixel_resolution'] = {'top-left': {'x':float(info[0]),
                                                                  'y':float(info[1])},
                                                     'top-right':{'x':float(info[2]),
                                                                  'y':float(info[3])}}
                elif 'THRESHOLDS' in line:
                    pass
                else:
                    return data_spec
def _get_tracking_info(asc_fname):
    tracking_info = _get_data_spec(asc_fname)
    tracking_info['camera'] = _get_header(asc_fname)['CAMERA']
    return tracking_info
    
def _is_sys_msg(line):
    return any(['!V' in line,
               '!MODE' in line,
              ';' in line])

def _event_to_dataframe(event_list, column_names, eventtype_label):
        df = pd.DataFrame(event_list,
                          columns=column_names).sort_values('timestamp',ascending=True)
        df['type'] = eventtype_label
        df.replace('.', np.NaN, inplace=True)
        return df
    
def _message_to_dataframe(message_list):
    events_with_offset = []
    events_no_offset = []
    for event in message_list:
        if event[1].lstrip('-').isdigit():
            events_with_offset.append([event[0],event[1],' '.join(event[2:])])
        else:
            events_no_offset.append([event[0], np.nan, ' '.join(event[1:])])
    return _event_to_dataframe((events_with_offset + events_no_offset),
                        ['timestamp','offset','message'],
                        'Message')  
        
    
def _replace_missing_vals(line):
    '''return list where missing gaze data
       (indicated by '.') are replaced by np.nan.
       line (list): list of values for 1 sample'''
    return [np.nan if i == '.' else i for i in line]

def _parse_recording_blocks(asc_fname):
    with Path(asc_fname).open() as file:
        samples = []
        events = {'START':[], 'END':[], 'SAMPLES':[], 'EVENTS':[],
                  'ESACC':[], 'EBLINK':[], 'EFIX':[],
                  'MSG':[], 'INPUT':[], 'BUTTON': []}
        
        is_recording_block = False
        for line in file:
            if line.startswith('START'):
                is_recording_block = True
            if is_recording_block and not _is_sys_msg(line):
                line = _replace_missing_vals(line.split())
                if line[0].isdigit():  # Sample lines start with a number.
                    samples.append(line)
                elif line[0] in events.keys():
                        events[f'{line[0]}'].append(line[1:])
                if line[0] == 'END':
                    is_recording_block = False
        return samples, events
        
class raw_eyelink():
    
    def __init__(self, asc_fname):
        self.asc_fname = Path(asc_fname)
        self.tracking_info = _get_tracking_info(self.asc_fname)
        self.session_info = _read_asc_header(self.asc_fname)
        
    def parse(self):
        with self.asc_fname.open() as file:
            samples = [] 
            events = {'trial_ids':[]}

            is_recording_block = False
            for line in file:
                if line.isspace() or  _is_system_message(line):
                    continue
                line = line.split()
                if 'TRIALID' in line:
                    events['trial_ids'].append([line[1], ' '.join(line[2:])])
                if line[0] == 'START':
                    is_recording_block = True
                if is_recording_block:
                    if line[0].isdigit():  # Sample lines start with a number.
                        samples.append(line)
                    elif line[0].isupper():  # Event strings are all-caps
                        if line[0] not in events.keys():
                            events[f'{line[0]}'] = [line[1:]]
                        else: # append line to existing key
                            events[f'{line[0]}'].append(line[1:])                
                if line[0] == 'END':
                    is_recording_block = False
            return samples, events
    
    def return_dataframes(self):
        dataframes = {}
        tracking_mode = self.tracking_info['tracking_mode']
        srate = self.tracking_info['srate']
        eyes_tracked = self.tracking_info['eyes_tracked']
        samples, events = self.parse()
    
        # SAMPLE cols
        input_port = False
        resolution = False
        velocity = False
        
        if 'INPUT' in events['SAMPLES'][0]:
            input_col = True
        
        input_col = ['serial_port_input']
        res_cols = ['x_res','y_res']
        flags_col = ['flags']
        
        timestamp_col = ['timestamp']
        binocular_cols = ['timestamp','x_pos_left','y_pos_left','pupil_size_left',
                          'x_pos_right','y_pos_right','pupil_size_right']
        monocular_cols = ['timestamp','x_pos','y_pos','pupil_size']
        
        remote_cols = ['head_target_x', 'head_target_y','head_target_distance', 'remote_flags']
        
        # EVENT cols
        event_timestamp_cols = ['eye','timestamp','end_timestamp','duration']
        fix_event_cols = ['fix_avg_x','fix_avg_y','fix_avg_pupil_size']
        sacc_event_cols = ['sacc_start_x','sacc_start_y','sacc_end_x','sacc_end_y','sacc_visual_angle','peak_velocity']
        
        
        if eyes_tracked == 'LR':   
            cols = binocular_cols
        elif eyes_tracked == 'L' or 'R':
            cols = monocular_cols

        if 'RES' in events['SAMPLES'][0]:
            cols = cols + res_cols
            fix_event_cols += res_cols
            sacc_event_cols += res_cols
        if 'INPUT' in events['SAMPLES'][0]:
            cols = cols + input_col
            
        cols = cols + flags_col
        
        if 'HTARGET' in events['SAMPLES'][0]:
            cols = cols + remote_cols
            
        dataframes['samples'] = pd.DataFrame(samples,columns=cols).sort_values('timestamp',ascending=True)     
        dataframes['samples'].replace('.', np.NaN, inplace=True) # missing gaze samples are '.'
        dataframes['samples']['type'] = 'gaze'
                                                                
        for eye_event, columns, label in zip(['EFIX','ESACC','EBLINK'],
                                             [event_timestamp_cols + fix_event_cols,
                                              event_timestamp_cols + sacc_event_cols,
                                              event_timestamp_cols],
                                             ['fixation','saccade','blink']):
            if eye_event in events:
                dataframes[label + 's'] = _event_to_dataframe(events[eye_event], columns, label)
        
        if 'MSG' in events:
            dataframes['messages'] = _message_to_dataframe(events['MSG'])
        dataframes['trial_ids'] = pd.DataFrame(events['trial_ids'], columns=['timestamp','trial'])
            
        #df_merged = pd.concat([*[events_dataframes[eye_event]
        #                       for eye_event in events_dataframes.keys()]
        #                      ])

        #df_eyelink = df_merged.sort_values('time_stamp')#df_merged[cols_orderd].sort_values('time_stamp')
        return dataframes

In [287]:
def _replace_missing_vals(line):
    '''return list where missing gaze data
       (indicated by '.') are replaced by np.nan'''
    return [np.nan if i == '.' else i for i in line]

def _parse_recording_blocks(asc_fname):
    with Path(asc_fname).open() as file:
        samples = [] 
        events = {}
        is_recording_block = False
        for line in file:
            if line.startswith('START'):
                is_recording_block = True
            if is_recording_block:
                if _is_system_message(line):
                    continue
                line = line.split()
                if line[0].isdigit():  # Sample lines start with a number.
                    samples.append(_replace_missing_vals(line))
                elif line[0].isupper():  # Event strings are all-caps
                    if line[0] in events.keys():  # append line to existing key
                        events[f'{line[0]}'].append(_replace_missing_vals(line[1:]))
                    else:
                        events[f'{line[0]}'] = [_replace_missing_vals(line[1:])]
                if line[0] == 'END':
                    is_recording_block = False
        return samples, events

In [288]:
samples, events = _parse_recording_blocks('s04s07_AS_18Feb22.asc')

In [289]:
events.keys()

dict_keys(['START', 'PRESCALER', 'VPRESCALER', 'PUPIL', 'EVENTS', 'SAMPLES', 'INPUT', 'SSACC', 'SBLINK', 'MSG', 'BUTTON', 'EBLINK', 'ESACC', 'SFIX', 'EFIX', 'END'])

In [187]:
plr_diksha = raw_eyelink('s04s07_PLR_18Feb22.asc')
plr_diksha.tracking_info

{'tracking_mode': 'corneal reflection',
 'srate': 1000,
 'sample_filter': 'extra filter',
 'analog_filter': 'standard filter',
 'eyes_tracked': 'L',
 'mount_config': 'I think Desktop (Remote mode), Target Sticker, Binocular/Monocular',
 'pixel_resolution': {'top-left': {'x': 0.0, 'y': 0.0},
  'top-right': {'x': 1919.0, 'y': 1079.0}},
 'camera': 'Eyelink GL Version 1.2 Sensor=AG7'}

In [150]:
plr_diksha.session_info

{'DATE': 'Fri Feb 18 08:21:04 2022',
 'TYPE': 'EDF_FILE BINARY EVENT SAMPLE TAGGED',
 'VERSION': 'EYELINK II 1',
 'SOURCE': 'EYELINK CL',
 'CAMERA': 'Eyelink GL Version 1.2 Sensor=AG7',
 'SERIAL NUMBER': 'CLG-BFE01',
 'CAMERA_CONFIG': 'BFE01200.SCD',
 'SREB2.2.299 WIN32 LID': '96F1E67 Mod:2022.02.19 00:25 EST'}

In [154]:
plr_diksha = raw_eyelink('s04s07_PLR_18Feb22.asc')
dfs = plr_diksha.return_dataframes()
dfs.keys()

dict_keys(['samples', 'fixations', 'saccades', 'blinks', 'messages', 'trial_ids'])

In [82]:
trial_idx = 0
dfs['messages']['trial'] = ''
for index, row in dfs['messages'].iterrows():
    if int(row['timestamp']) > int(events['trial_ids'][trial_idx + 1][0]):
        row['trial'] = events['trial_ids'][trial_idx][1]
    else:
        row['trial'] = events['trial_ids'][trial_idx + 1][1]
        try:
            trial_idx += 1
        except IndexError:
            continue


IndexError: list index out of range

In [136]:
antisaccade_diksha = raw_eyelink('s04s07_AS_18Feb22.asc')
dfs = antisaccade_diksha.return_dataframes()
dfs['trial_ids']

['CR', '1000', '2', '1', 'L']


,timestamp,trial
0,2726840,TRIALID 0
1,2732373,TRIALID 1
2,2737473,TRIALID 2
3,2742901,TRIALID 3
4,2747216,TRIALID 4
5,2751673,TRIALID 5
6,2755894,TRIALID 6
7,2760157,TRIALID 7
8,2764508,TRIALID 8
9,2768567,TRIALID 9


In [251]:
samples, events = antisaccade_diksha.parse()

In [189]:
antisaccade_diksha.tracking_info

{'tracking_mode': 'corneal reflection',
 'srate': '1000',
 'sample_filter': 'extra filter',
 'analog_filter': 'standard filter',
 'eyes_tracked': 'L',
 'mount_config': 'I think Desktop (Remote mode), Target Sticker, Binocular/Monocular'}

In [200]:
def _specify_missing_vals(sample_list):
    '''return new list where missing gaze data
       (indicated by '.') are replaced by np.nan'''
    return [[np.nan if i == '.' else i for i in line] for line in sample_list]
    

In [201]:
specify_missing_vals(samples)

[['2726896', nan, nan, '0.0', '1.0', '...'],
 ['2726897', nan, nan, '0.0', '1.0', '...'],
 ['2726898', nan, nan, '0.0', '1.0', '...'],
 ['2726899', nan, nan, '0.0', '1.0', '...'],
 ['2726900', nan, nan, '0.0', '1.0', '...'],
 ['2726901', nan, nan, '0.0', '1.0', '...'],
 ['2726902', nan, nan, '0.0', '1.0', '...'],
 ['2726903', nan, nan, '0.0', '1.0', '...'],
 ['2726904', nan, nan, '0.0', '1.0', '...'],
 ['2726905', nan, nan, '0.0', '1.0', '...'],
 ['2726906', nan, nan, '0.0', '1.0', '...'],
 ['2726907', nan, nan, '0.0', '1.0', '...'],
 ['2726908', nan, nan, '0.0', '1.0', '...'],
 ['2726909', nan, nan, '0.0', '1.0', '...'],
 ['2726910', nan, nan, '0.0', '1.0', '...'],
 ['2726911', nan, nan, '0.0', '1.0', '...'],
 ['2726912', nan, nan, '0.0', '1.0', '...'],
 ['2726913', nan, nan, '0.0', '1.0', '...'],
 ['2726914', nan, nan, '0.0', '1.0', '...'],
 ['2726915', nan, nan, '0.0', '1.0', '...'],
 ['2726916', nan, nan, '0.0', '1.0', '...'],
 ['2726917', nan, nan, '0.0', '1.0', '...'],
 ['2726918

In [38]:
events['trial_ids'][-1]

['2974407', 'TRIALID 51']

In [138]:
binocular = raw_eyelink('test_eyelink.asc')
samples, events = binocular.parse()
header = _read_asc_header('test_eyelink.asc')
dfs = binocular.return_dataframes()
dfs['samples']

,timestamp,x_pos_left,y_pos_left,pupil_size_left,x_pos_right,y_pos_right,pupil_size_right,flags,type
0,5511179,988.3,534.7,3879.0,989.5,513.6,3785.0,.....,gaze
1,5511181,987.0,536.3,3879.0,990.5,515.4,3782.0,.....,gaze
2,5511183,987.4,533.3,3868.0,989.7,512.5,3770.0,.....,gaze
3,5511185,988.2,531.2,3855.0,988.0,512.4,3783.0,.....,gaze
4,5511187,988.3,532.6,3858.0,987.8,514.4,3773.0,.....,gaze
...,...,...,...,...,...,...,...,...,...
30231,5571641,974.6,532.8,3477.0,962.0,554.8,3414.0,.....,gaze
30232,5571643,974.8,532.0,3484.0,963.8,554.3,3417.0,.....,gaze
30233,5571645,974.9,529.7,3477.0,963.6,551.9,3421.0,.....,gaze
30234,5571647,974.8,530.8,3487.0,962.0,550.3,3416.0,.....,gaze


In [140]:
binocular.tracking_info

{'tracking_mode': 'corneal reflection',
 'srate': '500',
 'sample_filter': 'extra filter',
 'analog_filter': 'standard filter',
 'eyes_tracked': 'LR',
 'mount_config': 'Desktop, Stabilized Head, Binocular/Monocular'}

In [34]:
no_blink = raw_eyelink('s01s05_VS_input.asc')
header = _read_asc_header('s01s05_VS_input.asc')
dfs = no_blink.return_dataframes()
dfs.keys()

dict_keys(['samples', 'fixations', 'saccades', 'messages', 'trial_ids'])

In [35]:
header

{'DATE': ' Fri Aug 27 07:38:36 2021',
 'TYPE': ' EDF_FILE BINARY EVENT SAMPLE TAGGED',
 'VERSION': ' EYELINK II 1',
 'SOURCE': ' EYELINK CL',
 'CAMERA': ' Eyelink GL Version 1.2 Sensor=AG7',
 'SERIAL NUMBER': ' CLG-BFE01',
 'CAMERA_CONFIG': ' BFE01200.SCD',
 'SREB2.2.299 WIN32 LID': '96F1E67 Mod:2021.04.09 22:49 EDT'}

In [32]:
monocular_res = raw_eyelink('scott2_VS_res.asc')
header = _read_asc_header('scott2_VS_res.asc')
dfs = monocular_res.return_dataframes()
dfs['samples']

,timestamp,x_pos,y_pos,pupil_size,x_res,y_res,serial_port_input,flags,type
0,1969018,953.8,541.9,3732.0,45.90,46.00,127.0,...,gaze
1,1969020,953.8,542.1,3734.0,45.90,46.00,127.0,...,gaze
2,1969022,953.9,542.3,3735.0,45.90,46.00,127.0,...,gaze
3,1969024,954.1,542.6,3737.0,45.90,46.00,127.0,...,gaze
4,1969026,954.4,543.1,3738.0,45.90,46.00,127.0,...,gaze
...,...,...,...,...,...,...,...,...,...
46078,2074190,1161.0,241.0,3231.0,46.10,45.50,127.0,...,gaze
46079,2074192,1161.2,241.4,3230.0,46.10,45.50,127.0,...,gaze
46080,2074194,1161.4,242.0,3230.0,46.10,45.50,127.0,...,gaze
46081,2074196,1161.6,242.6,3230.0,46.10,45.50,127.0,...,gaze


In [33]:
header

{'DATE': ' Fri Apr 23 08:16:25 2021',
 'TYPE': ' EDF_FILE BINARY EVENT SAMPLE TAGGED',
 'VERSION': ' EYELINK II 1',
 'SOURCE': ' EYELINK CL',
 'CAMERA': ' Eyelink GL Version 1.2 Sensor=AG7',
 'SERIAL NUMBER': ' CLG-BFE01',
 'CAMERA_CONFIG': ' BFE01200.SCD',
 'SREB2.2.299 WIN32 LID': '96F1E67 Mod:2021.04.09 22:49 EDT'}

In [12]:
eyelink_example = raw_eyelink('eyelink_sample_videoDV_AG.asc')
dfs = eyelink_example.return_dataframes()
dfs['messages']

,timestamp,offset,message,type
0,360814041,-8,Frame to be displayed 0,Message
1,360814083,-16,Frame to be displayed 1,Message
2,360814127,-6,Frame to be displayed 2,Message
3,360814169,-13,Frame to be displayed 3,Message
4,360814212,-4,Frame to be displayed 4,Message
...,...,...,...,...
519,360836604,-13,Frame to be displayed 519,Message
520,360836647,-4,Frame to be displayed 520,Message
521,360836690,-10,Frame to be displayed 521,Message
522,360836744,0,conditional,Message


In [84]:
head_target = raw_eyelink('PLR_head_target.asc')
dfs = head_target.return_dataframes()

In [89]:
dfs['samples']

,timestamp,x_pos,y_pos,pupil_size,flags,head_target_x,head_target_y,head_target_distance,remote_flags,type
141118,1000000,997.9,547.4,474.0,...,1087.0,5000.0,514.4,.............,gaze
141119,1000002,997.9,547.4,474.0,...,1087.0,5000.0,514.4,.............,gaze
141120,1000004,997.8,547.4,474.0,...,1086.0,5000.0,514.4,.............,gaze
141121,1000006,997.8,547.3,474.0,...,1086.0,5000.0,514.4,.............,gaze
141122,1000008,997.7,547.3,474.0,...,1086.0,5000.0,514.4,.............,gaze
...,...,...,...,...,...,...,...,...,...,...
141113,999990,997.8,546.5,473.0,...,1088.0,5000.0,514.5,.............,gaze
141114,999992,997.8,546.8,473.0,...,1088.0,5000.0,514.5,.............,gaze
141115,999994,997.8,547.0,473.0,...,1087.0,5000.0,514.5,.............,gaze
141116,999996,997.8,547.2,474.0,...,1087.0,5000.0,514.5,.............,gaze


In [93]:
dfs.keys()

dict_keys(['samples', 'fixations', 'saccades', 'blinks', 'messages', 'trial_ids'])

In [74]:
mount_config = {'MTABLER': 'Desktop, Stabilized Head, Monocular',
                'BTABLER': 'Desktop, Stabilized Head, Binocular/Monocular',
                'RTABLER': 'Desktop (Remote mode), Target Sticker, Monocular',
                'RBTABLER': 'Desktop (Remote mode), Target Sticker, Binocular/Monocular',
                'ABRTABLE': 'I think Desktop (Remote mode), Target Sticker, Binocular/Monocular',
                'AMTABLER': 'Arm Mount, Stabilized Head, Monocular',
                'ARTABLER': 'Arm Mount (Remote mode), Target Sticker, Monocular',
                'BTOWER': 'Binocular Tower Mount, Stabilized Head, Binocular/Monocular',
                'TOWER': 'Tower Mount, Stabilized Head, Monocular',
                'MPRIM': 'Primate Mount, Stabilized Head, Monocular',
                'BPRIM': 'Primate Mount, Stabilized Head, Binocular/Monocular',
                'MLRR': 'Long-Range Mount, Stabilized Head, Monocular, Camera Level',
                'BLRR': 'Long-Range Mount, Stabilized Head, Binocular/Monocular, Camera Angled'}